# Utilisation du LLM préinstallé dans Ollama et suivi dans MLflow

## Création d'un Experiment dans MLflow pour suivre les différentes exécutions

In [2]:
import mlflow
mlflow.set_tracking_uri("http://mlflow-serveur:8080/")
mlflow.set_experiment("ollama")

<Experiment: artifact_location='mlflow-artifacts:/995425785044879997', creation_time=1725891041882, experiment_id='995425785044879997', last_update_time=1725891041882, lifecycle_stage='active', name='my-experiment', tags={}>

## Mise en place du LLM

In [7]:
# This example demonstrates defining a model directly from code.
# This feature allows for defining model logic within a python script, module, or notebook that is stored
# directly as serialized code, as opposed to object serialization that would otherwise occur when saving
# or logging a model object.
# This script defines the model's logic and specifies which class within the file contains the model code.
# The companion example to this, chain_as_code_driver.py, is the driver code that performs the  logging and
# loading of this model definition.

from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.llms import Ollama

import mlflow

mlflow.langchain.autolog()


# Return the string contents of the most recent message from the user
def extract_user_query_string(chat_messages_array):
    return chat_messages_array[-1]["content"]


# Return the chat history, which is is everything before the last question
def extract_chat_history(chat_messages_array):
    return chat_messages_array[:-1]


prompt = PromptTemplate(
    template="You are a hello world bot. Respond with a reply to the user's question that is fun and interesting to the user.  User's question: {question}",
    input_variables=["question"],
)

model = Ollama(verbose=True,
               base_url="http://ollama:11434",
               model="llama3",
               )

chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_user_query_string),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_chat_history),
    }
    | prompt
    | model
    | StrOutputParser()
)


## Question et remontée dans MLflow

In [8]:
question = {
    "messages": [
        {
            "role": "user",
            "content": "what is rag?",
        },
    ]
}
print(chain.invoke(question))

# IMPORTANT: The model code needs to call `mlflow.models.set_model()` to set the model,
# which will be loaded back using `mlflow.langchain.load_model` for inference.
mlflow.models.set_model(model=chain)


Rag is actually a type of fabric made from scrap cloth pieces. It's been around for centuries and has been used for everything from cleaning to fashion. But did you know that rag rugs were once so valuable that they were passed down as family heirlooms? Now that's one fancy rag!
